In [1]:
import warnings
warnings.filterwarnings('ignore')

# 제5장 회원 탈퇴를 예측하는 테크닉10

### 테크닉 41 : 데이터를 읽어들이고 이용데이터를 수정하자

In [18]:
import pandas as pd
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,종일,10500,일반,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,종일,10500,일반,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,야간,6000,일반,3.916667,4.0,6,1,1,2019-04-30,47


In [13]:
year_months = list(uselog_months['연월'].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["연월"] == year_months[i]]
    tmp.rename(columns = {'count':'count_0'}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months['연월'] == year_months[i-1]]
    del tmp_before['연월']
    tmp_before.rename(columns = {'count':'count_1'}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)
    
uselog.head()


,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


### 테크닉 42 : 탈퇴전월의 탈퇴고객데이터를 작성하자

In [14]:
# 탈퇴신청을 한 전월의 데이터가 중요함
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"] == 1]
exit_customer['exit_date'] = None

In [15]:
exit_customer['end_date'] = pd.to_datetime(customer['end_date'])

In [17]:
for i in range(len(exit_customer)):
    exit_customer['exit_date'].iloc[i] = exit_customer['end_date'].iloc[i] - relativedelta(months=1)

exit_customer['연월'] = pd.to_datetime(exit_customer['exit_date']).dt.strftime("%Y%m")
uselog["연월"] = uselog["연월"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "연월"], how="left")
print(len(uselog))
exit_uselog.head()

33851


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
exit_uselog = exit_uselog.dropna(subset=['name'])
print(len(exit_uselog))
exit_uselog.head()

1104


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


### 테크닉 43 : 지속회원의 데이터를 작성하자

In [22]:
conti_customer = customer.loc[customer['is_deleted']==0]
conti_uselog = pd.merge(uselog, conti_customer, on=['customer_id'], how='left')
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=['name'])
print(len(conti_uselog))

33851
27422


In [23]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201807,PL013000,9,9.0,XXXXXX,C01,M,2018-05-15,NaN,CA2,...,종일,10500.0,입회비반액할인,7.727273,7.0,9.0,6.0,1.0,2019-04-30,11.0
1,201805,TS037478,6,12.0,XXXX,C02,M,2018-01-01,NaN,CA1,...,주간,7500.0,일반,7.333333,7.5,12.0,5.0,1.0,2019-04-30,15.0
2,201807,PL042341,5,9.0,XXXXX,C03,M,2017-11-01,NaN,CA1,...,야간,6000.0,일반,5.833333,6.0,9.0,1.0,1.0,2019-04-30,17.0
3,201901,IK453330,5,7.0,XXXX,C03,M,2015-10-01,NaN,CA1,...,야간,6000.0,일반,4.500000,4.5,7.0,1.0,1.0,2019-04-30,42.0
4,201902,HI015767,4,3.0,XXXXX,C03,M,2015-12-01,NaN,CA1,...,야간,6000.0,일반,5.000000,4.5,9.0,2.0,1.0,2019-04-30,40.0


In [24]:
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
print(len(predict_data))


3946


### 테크닉 44 : 예측할 달의 재적기간을 작성하자

In [26]:
predict_data['period'] = 0
predict_data['now_date'] = pd.to_datetime(predict_data['연월'], format="%Y%m")
predict_data['start_date'] = pd.to_datetime(predict_data['start_date'])

for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)
    
predict_data.head()


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201807,PL013000,9,9.0,XXXXXX,C01,M,2018-05-15,NaN,CA2,...,7.727273,7.0,9.0,6.0,1.0,2019-04-30,11.0,NaN,1,2018-07-01
1,201805,TS037478,6,12.0,XXXX,C02,M,2018-01-01,NaN,CA1,...,7.333333,7.5,12.0,5.0,1.0,2019-04-30,15.0,NaN,4,2018-05-01
2,201807,PL042341,5,9.0,XXXXX,C03,M,2017-11-01,NaN,CA1,...,5.833333,6.0,9.0,1.0,1.0,2019-04-30,17.0,NaN,8,2018-07-01
3,201901,IK453330,5,7.0,XXXX,C03,M,2015-10-01,NaN,CA1,...,4.500000,4.5,7.0,1.0,1.0,2019-04-30,42.0,NaN,39,2019-01-01
4,201902,HI015767,4,3.0,XXXXX,C03,M,2015-12-01,NaN,CA1,...,5.000000,4.5,9.0,2.0,1.0,2019-04-30,40.0,NaN,38,2019-02-01


### 테크닉 45 : 결측치를 제거하자

In [27]:
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1               262
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [28]:
predict_data = predict_data.dropna(subset=['count_1'])
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2632
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2632
period                  0
now_date                0
dtype: int64

### 테크닉 46 : 문자열 변수를 처리할수 있도록 가공하자

In [30]:
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()


,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,입회비반액할인,종일,M,9.0,1.0,1,0.0
1,일반,주간,M,12.0,1.0,4,0.0
2,일반,야간,M,9.0,1.0,8,0.0
3,일반,야간,M,7.0,1.0,39,0.0
4,일반,야간,M,3.0,1.0,38,0.0


In [31]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_일반,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_야간,class_name_종일,class_name_주간,gender_F,gender_M
0,9.0,1.0,1,0.0,0,0,1,0,1,0,0,1
1,12.0,1.0,4,0.0,1,0,0,0,0,1,0,1
2,9.0,1.0,8,0.0,1,0,0,1,0,0,0,1
3,7.0,1.0,39,0.0,1,0,0,1,0,0,0,1
4,3.0,1.0,38,0.0,1,0,0,1,0,0,0,1


In [32]:
del predict_data["campaign_name_일반"]
del predict_data["class_name_야간"]
del predict_data["gender_M"]

predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_종일,class_name_주간,gender_F
0,9.0,1.0,1,0.0,0,1,1,0,0
1,12.0,1.0,4,0.0,0,0,0,1,0
2,9.0,1.0,8,0.0,0,0,0,0,0
3,7.0,1.0,39,0.0,0,0,0,0,0
4,3.0,1.0,38,0.0,0,0,0,0,0


### 테크닉 47 : 의사결정나무를 사용해서 탈퇴예측모델을 작성하자

In [39]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data['is_deleted']== 1]
conti = predict_data.loc[predict_data['is_deleted']== 0]

X = pd.concat([exit, conti], ignore_index=True)
y = X['is_deleted']
del X['is_deleted']

,count_1,routine_flg,period,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_종일,class_name_주간,gender_F
0,3.0,0.0,2,0,0,1,0,0
1,3.0,1.0,6,0,0,1,0,0
2,3.0,0.0,11,0,0,1,0,0
3,3.0,0.0,3,0,0,1,0,0
4,4.0,0.0,8,0,0,0,0,1
...,...,...,...,...,...,...,...,...
3679,8.0,1.0,1,0,0,0,0,1
3680,5.0,1.0,3,1,0,0,1,1
3681,6.0,1.0,0,0,0,1,0,1
3682,7.0,1.0,1,1,0,0,0,0


In [41]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)
print(len(X_train))
print(len(y_train))

2763
2763


In [42]:
model = DecisionTreeClassifier(random_state = 0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1.
 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0.
 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1.
 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0.
 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1.
 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1.
 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1.
 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1.
 0. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.

In [43]:
results_test = pd.DataFrame({"y_test":y_test, "y_pred":y_test_pred})
results_test.head()

,y_test,y_pred
3252,0.0,0.0
2373,0.0,0.0
900,1.0,1.0
1874,0.0,0.0
3349,0.0,0.0


### 테크닉 48 : 예측 모델을 평가하고, 모델을 튜닝해보자

In [44]:
correct = len(results_test.loc[results_test['y_test'] == results_test['y_pred']])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.9229098805646037


In [47]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9229098805646037
0.9786463988418386


In [49]:
X = pd.concat([exit, conti], ignore_index=True)
y = X['is_deleted']
del X['is_deleted']

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model2 = DecisionTreeClassifier(random_state=0, max_depth=5)
model2.fit(X_train, y_train)

print(model2.score(X_test, y_test))
print(model2.score(X_train, y_train))

0.9207383279044516
0.9420919290626131


### 테크닉 49 : 모델에 기여하고 있는 변수를 확인하자

In [48]:
importance = pd.DataFrame({"feature_name":X.columns, "coefficient":model.feature_importances_})
importance

,feature_name,coefficient
0,count_1,0.314045
1,routine_flg,0.396344
2,period,0.194265
3,campaign_name_입회비무료,0.021853
4,campaign_name_입회비반액할인,0.014059
5,class_name_종일,0.025232
6,class_name_주간,0.014817
7,gender_F,0.019387


In [50]:
importance = pd.DataFrame({"feature_name":X.columns, "coefficient":model2.feature_importances_})
importance

,feature_name,coefficient
0,count_1,0.350955
1,routine_flg,0.491452
2,period,0.154574
3,campaign_name_입회비무료,0.000434
4,campaign_name_입회비반액할인,0.000000
5,class_name_종일,0.000000
6,class_name_주간,0.000482
7,gender_F,0.002104


### 테크닉 50 : 회원의 탈퇴를 예측하자

In [59]:
count_1 = 4
routine_flg = 1
period = 10
campaign_name = "입회비무료"
class_name = "종일"
gender = "M"

In [60]:
if campaign_name == "입회비반값할인":
    campaign_name_list = [1,0]
elif campaign_name == "입회비무료":
    campaign_name_list = [0,1]
elif campaign_name == "일반":
    campaign_name_list = [0,0]

if class_name == "종일":
    class_name_list = [1,0]
elif class_name == "주간":
    class_name_list = [0,1]
elif class_name == "야간":
    class_name_list = [0,0]

if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]

input_data = [count_1, routine_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

input_data

[4, 1, 10, 0, 1, 1, 0, 0]

In [61]:
print(model.predict([input_data]))
print(model.predict_proba([input_data]))
print(model2.predict([input_data]))
print(model2.predict_proba([input_data]))

[1.]
[[0.25 0.75]]
[1.]
[[0.11688312 0.88311688]]
